In [ ]:
from datasets import load_dataset, Dataset
from torchdata.stateful_dataloader import StatefulDataLoader
from new_pipeline import process_data
from huggingface_hub import HfApi

/Users/nrh146/Documents/cc-genealogy/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset("allenai/c4", "en", split="train", streaming=True)
ds = ds.remove_columns(["text", "timestamp"])
ds = ds.map(process_data)
ds = ds.with_format("torch")

In [34]:
dataloader = StatefulDataLoader(ds, batch_size=100, num_workers=4)

In [35]:
# Iterate through the streaming dataset using the dataloader
# We'll get batches of 100 examples each
for i, batch in enumerate(dataloader):
    # Print a sample of the batch
    print(f"Batch {i+1}:")
    print(f"Number of examples in batch: {len(batch['url'])}")
    print(f"Sample url: {batch['url'][0][:100]}...")  # Show first 100 chars of first example
    
    # Optional: process or save your batch here
    
    # Break after a few batches to avoid endless iteration
    if i >= 4:  # Show 5 batches
        break

Batch 1:
Number of examples in batch: 100
Sample url: https://klyq.com/beginners-bbq-class-taking-place-in-missoula/...
Batch 2:
Number of examples in batch: 100
Sample url: http://sevenavedesign.com/studio/...
Batch 3:
Number of examples in batch: 100
Sample url: http://help.browntape.com/support/solutions/articles/1000176729-adding-myntra-as-a-channel...
Batch 4:
Number of examples in batch: 100
Sample url: http://www.excellence-in-stress-management.eu/surveys/...
Batch 5:
Number of examples in batch: 100
Sample url: https://cdpug.org/free-breakfast-lecture-series-for-the-creative-community/...


In [2]:
ds = load_dataset("allenai/c4", "en", split="train", streaming=True)
ds = ds.remove_columns(["text", "timestamp"])
ds = ds.map(process_data, batched=True, batch_size=1_000)

In [ ]:
ds = load_dataset("allenai/c4", "en", split="train", streaming=True)
ds = ds.remove_columns(["text", "timestamp"])
ds = ds.map(process_data, batched=True, batch_size=1_000)

api = HfApi()
repo_id = "nhagar/test-upload-c4"
api.create_repo(repo_id, exist_ok=True, repo_type="dataset")

chunk_size = 100_000
for i, chunk_ds in enumerate(ds.iter(batch_size=chunk_size)):
    chunk_regular = Dataset.from_dict(chunk_ds)

    chunk_regular.push_to_hub(
        repo_id=repo_id,
        data_dir=f"chunks/chunk_{i}",
    )

RepoUrl('https://huggingface.co/datasets/nhagar/test-upload-c4', endpoint='https://huggingface.co', repo_type='dataset', repo_id='nhagar/test-upload-c4')

In [ ]:
chunk_size = 100_000
for i, chunk_ds in enumerate(ds.iter(batch_size=chunk_size)):
    chunk_regular = Dataset.from_dict(chunk_ds)

    chunk_regular.push_to_hub(
        repo_id=repo_id,
        data_dir=f"chunks/chunk_{i}",
    )

Uploading the dataset shards: 100%|██████████| 1/1 [00:04<00:00,  4.63s/it]


In [ ]:
ds = load_dataset("allenai/c4", "en", split="train", streaming=True)
ds = ds.remove_columns(["text", "timestamp"])
ds = ds.map(process_data, batched=True, batch_size=1_000)

api = HfApi()
repo_id = "nhagar/test-upload-c4"
api.create_repo(repo_id, exist_ok=True, repo_type="dataset")

chunk_size = 1_000_000
for i, chunk_ds in enumerate(ds.iter(batch_size=chunk_size)):
    chunk_regular = Dataset.from_dict(chunk_ds)

    chunk_regular.push_to_hub(
        repo_id=repo_id,
        data_dir=f"chunks/chunk_{i}",
    )

In [24]:
# Initialize a counter for total number of examples processed
total_examples = 0
i = 0

processed_data = []

# Loop through dataset until we reach or exceed 1 million examples
for i, example in enumerate(ds):
    print(example)
    # Process each example
    total_examples += len(example['url'])
    
    # Print progress every 100,000 examples
    if i % 10000 == 0:
        print(f"Batch {i}: Total examples so far: {total_examples}")

    processed_data["url"].extend(example["url"])
    processed_data["tld"].extend(example["tld"])

    if len(processed_data["url"]) >= 500_000:
        print("Dumping data")
        processed_data = []
    
    # Break once we've reached 1 million examples
    if total_examples >= 10:
        print(f"Reached target: {total_examples} examples processed in {i+1} batches")
        break

print(f"Final count: {total_examples} examples across {i+1} batches")

{'url': 'https://klyq.com/beginners-bbq-class-taking-place-in-missoula/', 'tld': 'klyq.com'}
Batch 0: Total examples so far: 62


TypeError: list indices must be integers or slices, not str

In [23]:
example

{'url': 'https://worldfinance100.com/2013/intelec-holdings/',
 'tld': 'worldfinance100.com'}

In [9]:
ds = load_dataset("allenai/c4", "en", split="train", streaming=True)
ds = ds.remove_columns(["text", "timestamp"])
ds = ds.map(process_data, batched=True, batch_size=10_000)
ds.push_to_hub("nhagar/proc-test-c4", private=False)

AttributeError: 'IterableDataset' object has no attribute 'push_to_hub'